# Calculate Average Annual Growth Rate
__Earnings comparison: Finance vs Engineering and Medical Research__

[US Census Bureau - how to adjust for inflation](https://www.census.gov/topics/income-poverty/income/guidance/current-vs-constant-dollars.html#:~:text=Example%3A%20To%20use%20the%20CPI,RS%20from%201995%20(225.3).&text=Current%20dollars%20is%20a%20term,household%2C%20or%20family%20receives%20it.)

[CPI python package](https://github.com/datadesk/cpi#:~:text=Adjusting%20for%20inflation%20is%20as,the%20Bureau%20of%20Labor%20Statistics.)

[Average Annual Growth Rates](https://www.investopedia.com/terms/a/aagr.asp#:~:text=The%20average%20annual%20growth%20rate,a%20series%20of%20growth%20rates.)

In [47]:
import pandas as pd
import blspandas
import api_key
import requests
import json
from datetime import datetime

### Query the data from the bureau of labor statistics

In [5]:
bls_dict = {
    'CES6054171003': 'Engineering and Medical Research',
    'CES5552300003': 'Financial Services'
}

# API key in config.py which contains: bls_key = 'key'.
key = '?registrationkey={}'.format(api_key.bls_key)

# Date Range
dates = ('2010', '2020')

# Query the BLS API.
query = blspandas.query_bls(bls_dict,key,dates)
query.head()

,Engineering and Medical Research,Financial Services
2010-01-01,41.96,41.98
2010-02-01,42.19,42.00
2010-03-01,41.97,42.16
2010-04-01,41.95,41.71
2010-05-01,41.99,41.69


In [16]:
query.tail()

,Engineering and Medical Research,Financial Services
2019-08-01,52.28,54.44
2019-09-01,52.12,54.04
2019-10-01,52.49,54.03
2019-11-01,52.67,54.18
2019-12-01,52.92,54.26


## Average Annual Growth Rate from 2010 to 2019
[Investopedia](https://www.investopedia.com/terms/a/aagr.asp#:~:text=The%20average%20annual%20growth%20rate,a%20series%20of%20growth%20rates.)
<br>
__Calculated by finding the arithmetic mean of the growth rate.__

We will calculate the __inflation-adjusted__ average growth rate following [this article's](https://www.census.gov/topics/income-poverty/income/guidance/current-vs-constant-dollars.html#:~:text=Example%3A%20To%20use%20the%20CPI,RS%20from%201995%20(225.3).&text=Current%20dollars%20is%20a%20term,household%2C%20or%20family%20receives%20it.) recommendation.
<br>

### 1. Adjust for inflation

In [55]:
# Make a copy of the dataset and reset the index and rename to Date
df = query.copy().reset_index().rename(columns={'index':'Date'})

# Change the units to yearly
df['Engineering and Medical Research'] = df['Engineering and Medical Research']*2080
df['Financial Services'] = df['Financial Services'] *2080

# Add a year column
df['Year'] = pd.to_datetime(df.Date).dt.year
df['Month'] = pd.to_datetime(df.Date).dt.month

# Subset only from 2010 to 2018
df = df.query('Year <= 2018')

# Adjust for inflation

def adjust_inflation(year):
    
    cpi_index = pd.read_csv('cpi_index.csv')
    
    cpi_2018 = cpi_index[cpi_index.Year == 2018][['CPI-U-RS Index']].values[0][0]
    
    cpi_year = cpi_index[cpi_index.Year == year][['CPI-U-RS Index']].values[0][0]
    
    cpi_ratio = cpi_2018 / cpi_year
    
    return cpi_ratio

# Create a column for the inflation adjustment ratio - in 2018 dollars
df['Inflation adjustment'] = df['Year'].apply(adjust_inflation)

# Create a new column by multiplying the salary by the inflation ratio
df['Engineering and Research salary - inflation adjusted'] = df['Engineering and Medical Research'] * df['Inflation adjustment']
df['Financial Services salary average - inflation adjusted'] = df['Financial Services'] * df['Inflation adjustment']

# Query only December Values
df = df.query('Month == 12')

df['Engineering_pct_change'] = round(df['Engineering and Research salary - inflation adjusted'].pct_change(),4)
df['Finance_pct_change'] = round(df['Financial Services salary average - inflation adjusted'].pct_change(),4)

df
# df.to_csv('Inflation Adjusted Percent Changes.csv')

,Date,Engineering and Medical Research,Financial Services,Year,Month,Inflation adjustment,Engineering and Research salary - inflation adjusted,Financial Services salary average - inflation adjusted,Engineering_pct_change,Finance_pct_change
11,2010-12-01,88961.6,85259.2,2010,12,1.154182,102677.901623,98404.657179,NaN,NaN
23,2011-12-01,90480.0,88046.4,2011,12,1.118911,101239.043873,98516.062693,-0.0140,0.0011
35,2012-12-01,93246.4,95867.2,2012,12,1.095704,102170.425837,105042.046104,0.0092,0.0662
47,2013-12-01,94411.2,98051.2,2013,12,1.079708,101936.530686,105866.667912,-0.0023,0.0079
59,2014-12-01,96304.0,100152.0,2014,12,1.061728,102248.691358,106334.222222,0.0031,0.0044
71,2015-12-01,98363.2,100547.2,2015,12,1.059903,104255.406592,106570.233763,0.0196,0.0022
83,2016-12-01,101961.6,103958.4,2016,12,1.046406,106693.264516,108782.728693,0.0234,0.0208
95,2017-12-01,102003.2,110219.2,2017,12,1.024377,104489.704598,112905.983823,-0.0207,0.0379
107,2018-12-01,105164.8,112590.4,2018,12,1.000000,105164.800000,112590.400000,0.0065,-0.0028


### 2. Calculate Average Annual Growth Rate

In [53]:
print('Engineering - AAGR: ', round(df['Engineering_pct_change'].mean()*100,2))
print('Finance - AAGR: ', round(df['Finance_pct_change'].mean()*100,2))

Engineering - AAGR:  0.31
Finance - AAGR:  1.72


# Group by year to find average salary

In [14]:
df_g = df.groupby('Year').mean().reset_index()
df_g_melted = pd.melt(df_g,
                      id_vars=['Year'],
                      value_vars=df_g.columns[1:3],
                      var_name='Industry',
                      value_name='Hourly Pay')
df_g_melted['Average Salary'] = df_g_melted['Hourly Pay']*2080
df_g_melted

,Year,Industry,Hourly Pay,Average Salary
0,2010,Engineering and Medical Research,42.078333,87522.933333
1,2011,Engineering and Medical Research,43.121667,89693.066667
2,2012,Engineering and Medical Research,44.450833,92457.733333
3,2013,Engineering and Medical Research,45.207500,94031.600000
4,2014,Engineering and Medical Research,46.055833,95796.133333
5,2015,Engineering and Medical Research,46.863333,97475.733333
6,2016,Engineering and Medical Research,48.071667,99989.066667
7,2017,Engineering and Medical Research,48.982500,101883.600000
8,2018,Engineering and Medical Research,49.763333,103507.733333
9,2019,Engineering and Medical Research,51.812500,107770.000000


## Average Annual Growth Rate

In [9]:
avg_growth_rate = df_g_melted.copy()
avg_growth_rate['Salary Growth'] = avg_growth_rate['Average Salary'].diff()
avg_growth_rate

,Year,Industry,Hourly Pay,Average Salary,Salary Growth
0,2010,Engineering and Medical Research,42.078333,87522.933333,NaN
1,2011,Engineering and Medical Research,43.121667,89693.066667,2170.133333
2,2012,Engineering and Medical Research,44.450833,92457.733333,2764.666667
3,2013,Engineering and Medical Research,45.207500,94031.600000,1573.866667
4,2014,Engineering and Medical Research,46.055833,95796.133333,1764.533333
5,2015,Engineering and Medical Research,46.863333,97475.733333,1679.600000
6,2016,Engineering and Medical Research,48.071667,99989.066667,2513.333333
7,2017,Engineering and Medical Research,48.982500,101883.600000,1894.533333
8,2018,Engineering and Medical Research,49.763333,103507.733333,1624.133333
9,2019,Engineering and Medical Research,51.812500,107770.000000,4262.266667


In [47]:
# df_g_melted.to_csv('Average Salary BY YEAR comparison.csv',index=False)

In [28]:
# df_g.to_csv('Earnings comparison BY YEAR - Engineering and Medical research vs Financial Services.csv')

In [41]:
earnings_growth = df_g[df_g.Year.isin([2010,2019])]
earnings_growth = pd.DataFrame(earnings_growth.diff(axis=0).iloc[1,:])
earnings_growth = earnings_growth.transpose().iloc[:,1:]
# earnings_growth.to_csv('Earnings GROWTH comparison from 2010 to 2019.csv',index=False)

In [48]:
earnings_growth

,Engineering and Medical Research,Financial Services,Research percent change,Financial services percent change
9,9.734167,12.808333,0.002061,0.002061
